# Transfer Learning
Using VGG16

In [ ]:
import tensorflow as tf
import keras
print('tensorflow version: ', tf.__version__)
print('keras version: ', keras.__version__)

tensorflow version:  2.9.2
keras version:  2.9.0


In [ ]:
# define sess to use gpu
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Quadro P4000, pci bus id: 0000:00:05.0, compute capability: 6.1



In [ ]:
# import necessary libraries and functions
import os
import joblib
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import Dense, InputLayer
from keras.models import Sequential

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import sys

sys.path.append('..')
from utils.s3_class import S3Functions
s3_funcs = S3Functions(bucket_name='jdgallegoq-pinacle')

In [ ]:
# create random number generator
seed = 42
rng = np.random.RandomState(seed)

In [ ]:
# train data
train_data = pd.read_csv(s3_funcs.read_object(key='gender_clf/train/train.csv'))
print(train_data.info())

# test data
test_data = pd.read_csv(s3_funcs.read_object(key='gender_clf/test.csv'))
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12196 entries, 0 to 12195
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_names  12196 non-null  object
 1   class        12196 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 190.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5227 entries, 0 to 5226
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_names  5227 non-null   object
dtypes: object(1)
memory usage: 41.0+ KB
None


In [ ]:
train_data['class'].value_counts()

1    6174
0    6022
Name: class, dtype: int64

In [ ]:
# load images
IMAGES_PATH = 'gender_clf/train/images/'

X = []
not_found = []
num_images_read = 5000
#for img_name in tqdm(train_data.image_names):
for img_name in tqdm(train_data.image_names[:num_images_read]):
  img = s3_funcs.read_image(key=IMAGES_PATH+img_name)
  X.append(img)

X = np.array(X)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
# Model architecture
from keras.models import Model
from keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.layers import (
    InputLayer,
    Dense
    )
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
# preprocess according to pretained model parameters
X = preprocess_input(X, mode='tf')

# define target as categorical (VGG16 target type)
y = to_categorical(y)

In [ ]:
# split dataset
y = train_data['class'].values[:num_images_read]
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=seed
)

del X

In [ ]:
# Load architecture
# VGG16 was pretrained on MNIST and ImageNet
# so load ImageNet weights cause our clf problem
# is more suitable with it.
base_model = VGG16(weights='imagenet')

# show summary
base_model.summary()

In [ ]:
# we're going to create a new model cause
# we need only 2 (only 2 categories) output not 1000.
# so, let's use all weights insted of last layers
x = Dense(100, activation='relu', name='my_dense')(base_model.layers[-4].output)
y = Dense(2, activation='softmax', name='my_pred')(x)
my_model = Model(input=base_model.input, output=y)
my_model.summary()

In [ ]:
# compile the model
my_model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'],
)

In [ ]:
# train
epochs = 10
my_model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data=(X_valid, y_valid)
)

In [ ]:
# validate metrics
preds = my_model.predict(X_valid)
predicted_class = np.argmax(preds, axis=1)

accuracy_score(y_valid, predicted_class)